In [3]:
# !curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt --output "input.txt"

In [4]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    
print("length of dataset: ", len(text))

length of dataset:  1115394


In [5]:
# make a list of all unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print("Vocab size: ",vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size:  65


In [6]:
# create mapping from character to index and index to character
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }

encode = lambda input_string: [stoi[char] for char in input_string] # Encoder: take a string as input, output a list of integer
decode = lambda int_list: ''.join([itos[i] for i in int_list]) # Decoder: take a list of integer as input, output a string

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [7]:
import torch

# Encode entire dataset and save into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)

print("Data shape and type: " , data.shape, data.dtype)

Data shape and type:  torch.Size([1115394]) torch.int64


C:\Users\hungd\AppData\Local\Temp\ipykernel_17772\2297166899.py:4: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xe . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  data = torch.tensor(encode(text), dtype=torch.long)


In [8]:
# Split the data into train and validation sets (90:10)
n = int(0.9 * len(text))
train_data = data[:n]
val_data = data[n:]

In [9]:
torch.manual_seed(20)

batch_size = 4 # number of independent sequences to be processed parallely 
block_size = 8 # maximum context length for predictions

def get_batch(split: str) -> torch.Tensor:
    """Generate a small batch of data including inputs x and targets y

    Args:
        split (str): "train" or "val"
    """
    data = train_data if split == "train" else val_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: block_size+i+1] for i in ix])
    
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)


inputs:
torch.Size([4, 8])
tensor([[40, 43, 56, 58,  5, 57,  1, 42],
        [26, 53, 61,  1, 61, 43, 56, 43],
        [57, 49,  1, 63, 53, 59, 56,  1],
        [ 1, 57, 59, 47, 58, 11,  0, 13]])
targets:
torch.Size([4, 8])
tensor([[43, 56, 58,  5, 57,  1, 42, 39],
        [53, 61,  1, 61, 43, 56, 43,  1],
        [49,  1, 63, 53, 59, 56,  1, 46],
        [57, 59, 47, 58, 11,  0, 13, 52]])


In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(20)

class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        """Calculate logits and loss from inputs and targets
        B for batch size, T for block size
        
        Args:
            idx (_type_): (B, T) tensor of integers as input. 
            targets (_type_, optional): (B, T) tensor of interger as corresponding target value. Defaults to None.
        """
        
        logits = self.token_embedding_table(idx) # return a tensor of shape (B, T, C) C as number of channel
        
        if targets == None:
            loss = None
        else:        
            # cross_entropy function of Pytorch take input with a specific shape so need to reshape logits and targets
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is the array of idices in current context
        for _ in range(max_new_tokens):
            # predictions
            logits, loss = self(idx)
            
            # get only the last time step
            logits = logits[:, -1, :] # shape (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx
            
        
m = BiagramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.6764, grad_fn=<NllLossBackward0>)


In [11]:
idx = torch.zeros((1,1), dtype=torch.long)
new_tokens = 100
output = m.generate(idx = idx, max_new_tokens=new_tokens)[0].tolist()

print(decode(output))


-vSVFI!j?C3jfCsmF O:iudFX!?F;$wuowmYtwNxHbN
zePdHvTeooS$UxZJnIFjlh3rHvYG
f!di
E-?Qu?FLkRgbcz
twm,ErQ


In [12]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [13]:
batch_size = 32

for steps in range(15000):
    xb, yb = get_batch('train')
    
    #evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.3691916465759277


In [14]:
idx = torch.zeros((1,1), dtype=torch.long)
new_tokens = 500
output = m.generate(idx = idx, max_new_tokens=new_tokens)[0].tolist()

print(decode(output))



KEayowieea ss tar, met theshiurearrde'e utousig shars trnduithit hencce pertole t hatlfutth r prldee fekedvesowave:
S:
'CERDUSeane othituge stheck d my de ind t II wen cancck:
MEENEThtha thw st fowind mputcozedshour t t lld!
Wharor d,
He farreronunls, hendele! l bo y s nghe nder!
Nake cke fim nnoun, w?
Th Ito t heaje wimedeso f thty l:
YOKINTI ge came.

GHeledut pea mm tno thithee s thit wow grt f ghmpuar! mes t catuknghe,
Thy frwarur wer d e t, o s
Pem msthin hathe, O:
Ged athitr y, bouns;

BO


## The mathematical trick in self-attention

In [15]:
# Calculate running average on each column of a tensor
torch.manual_seed(20)

a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)

b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print("a = ")
print(a)
print("b = ")
print(b)
print("c = ")
print(c)


a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b = 
tensor([[1., 0.],
        [7., 3.],
        [4., 0.]])
c = 
tensor([[1.0000, 0.0000],
        [4.0000, 1.5000],
        [4.0000, 1.0000]])


In [16]:
# set up example
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [17]:
# x[b, t] = mean_{i <= t} x[b, i]
# Version 1
xbow = torch.zeros((B, T, C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # shape (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [18]:
# Version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) ---> torch will automatically convert (T, T) to (B, T, T) ---> (B, T, T) @ (B, T, C) => (B, T, C)

In [19]:
# Version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T)) # affinities between token, set to be zeros in the beginning
wei = wei.masked_fill(tril == 0, float('-inf')) # make what comes in the future cannot affect the current calculation
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x

In [20]:
# Version 4: Self-attention
torch.manual_seed(20)
B, T, C = 4, 8, 32 #batch, time, channels
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones(T, T))

wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [22]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1)

print(k.var())
print(q.var())
print(wei.var())

tensor(1.0908)
tensor(0.9813)
tensor(16.2205)


In [30]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

print(k.var())
print(q.var())
print(wei.var())

tensor(0.8489)
tensor(1.0486)
tensor(0.9213)


In [31]:
'cuda' if torch.cuda.is_available() else 'cpu'

'cuda'